In [1]:
import cv2
import torch
from ultralytics import YOLO

def detect_coke_bottle(image_path, model_path, real_height_cm=24):
    """
    Detects a  Coca-Cola bottle and calculates its real-world height.
    :param image_path: Path to the image.
    :param model_path: Path to the trained YOLO model.
    :param real_height_cm: Known real-world height of the  Coca-Cola bottle.
    :return: Bounding box height in pixels and estimated real-world height.
    """
    # Load the YOLO model
    model = YOLO(model_path)
    
    # Read the image
    image = cv2.imread(image_path)
    
    # Perform object detection
    results = model(image)
    
    # Get the detected bounding boxes
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0]  # Get bounding box coordinates
            height_pixels = y2 - y1  # Calculate height in pixels
            
            # Calculate Pixel-to-CM ratio
            pixel_to_cm_ratio = real_height_cm / height_pixels
            estimated_real_height = height_pixels * pixel_to_cm_ratio
            
            # print(f"Detected Coca-Cola Bottle Height: {height_pixels:.2f} pixels")
            # print(f"Estimated Real-World Height: {estimated_real_height:.2f} cm")
            
            return height_pixels, estimated_real_height
    
    print("No Coca-Cola bottle detected.")
    return None, None

# Example Usage
image_path = "image_folder/image5.jpeg"  # Change to your image path
model_path = "best (8).pt"  # Path to your trained YOLO model
height_pixels, estimated_real_height =detect_coke_bottle(image_path, model_path)

print(f"Detected Coca-Cola Bottle Height: {height_pixels:.2f} pixels")
print(f"Estimated Real-World Height: {estimated_real_height:.2f} cm")


0: 800x608 1 750ml coke, 372.4ms
Speed: 4.3ms preprocess, 372.4ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 608)
Detected Coca-Cola Bottle Height: 79.95 pixels
Estimated Real-World Height: 24.00 cm


In [2]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO

def load_model(model_path):
    """Loads the YOLO model."""
    return YOLO(model_path)

In [3]:
def detect_objects(model, image_path):
    """Detects objects in the given image."""
    image = cv2.imread(image_path)
    results = model(image)
    return results, image

In [4]:
def get_average_coordinate(results, target_class):
    """Finds the median y-coordinate of the detected class to stabilize detection."""
    y_coords = []
    for result in results:
        for box in result.boxes:
            cls = int(box.cls.cpu().numpy().item())
            if cls == target_class:
                y_min, y_max = int(box.xyxy[0][1]), int(box.xyxy[0][3])
                y_coords.append((y_min, y_max))
    
    if not y_coords:
        return None
    
    median_y_min = np.median([y[0] for y in y_coords])
    median_y_max = np.median([y[1] for y in y_coords])
    return median_y_min, median_y_max


In [5]:
def calculate_height(ceiling_y, floor_y, calibration_obj=None):
    """Calculates height in pixels or real-world units if a calibration object is given."""
    if ceiling_y is None or floor_y is None:
        print("Ceiling or Floor not detected correctly!")
        return None
    
    pixel_height = floor_y - ceiling_y
    
    if calibration_obj:
        real_height, obj_pixel_height = calibration_obj
        pixel_to_cm_ratio = real_height / obj_pixel_height
        real_ceiling_height = pixel_height * pixel_to_cm_ratio
        return real_ceiling_height
    
    return pixel_height  # Return only pixel height if no calibration object

In [6]:
def visualize(image, ceiling_y, floor_y):
    """Draws a vertical line from ceiling to floor for visualization."""
    img_height, img_width, _ = image.shape
    center_x = img_width // 2  # Midpoint in width

    cv2.line(image, (center_x, int(ceiling_y)), (center_x, int(floor_y)), (0, 255, 0), 2)  # Green line
    output_path = "output.jpg"
    cv2.imwrite(output_path, image)
    print(f"Image saved with vertical line at: {output_path}")

In [9]:
# Example usage
model_path = "best (5).pt"  # Path to trained YOLO model
  # Input image path
calibration_obj = (estimated_real_height, height_pixel)  # Example: (real-world height in cm, pixel height)

# Load model
model = load_model(model_path)
results, image = detect_objects(model, image_path)

# Get median ceiling and floor coordinates
ceiling_y, _ = get_average_coordinate(results, target_class=0)  # Assuming class 0 is ceiling
_, floor_y = get_average_coordinate(results, target_class=1)  # Assuming class 1 is floor

if ceiling_y is not None and floor_y is not None:
    pixel_height = floor_y - ceiling_y
    print(f"Pixel Height: {pixel_height} pixels")

    # Example calibration using a Coca-Cola bottle
    # calibration_obj = (24, 150)  # (Real-world height in cm, detected pixel height)
    height = calculate_height(ceiling_y, floor_y, calibration_obj)
    print(f"Ceiling Height: {height:.2f} cm")
    
    visualize(image, ceiling_y, floor_y)


0: 640x480 1 ceiling, 3 floors, 2 wallss, 321.3ms
Speed: 2.2ms preprocess, 321.3ms inference, 8.3ms postprocess per image at shape (1, 3, 640, 480)
Pixel Height: 1273.0 pixels
Ceiling Height: 321.60 cm
Image saved with vertical line at: output.jpg
